In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

In [6]:
# Загрузка данных
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='Price')

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение первой модели
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Base model score: MSE: {mse:.2f}, R^2: {r2:.2f}")

Base model score: MSE: 0.29, R^2: 0.78


In [ ]:
# Различные виды распределений
distributions = {
    'uniform': lambda num_samples: np.random.rand(num_samples),
    'normal': lambda num_samples: np.random.randn(num_samples),
    'exponential': lambda num_samples: np.random.exponential(size=num_samples),
    # 'gamma': lambda num_samples: np.random.gamma(shape=2.0, scale=1.0, size=num_samples)
}

num_samples = 1000
best_mse = float('inf')
best_r2 = float('-inf')
best_distribution = None
best_new_X = None
best_new_y = None

# Все возможные комбинации распределений для каждого столбца
distribution_names = list(distributions.keys())
all_combinations = list(itertools.product(distribution_names, repeat=X_train.shape[1]))

# Функция для генерации данных и оценки модели
def evaluate_combination(combination):
    random_data = np.zeros((num_samples, X_train.shape[1]))
    
    for col_idx, dist_name in enumerate(combination):
        random_data[:, col_idx] = distributions[dist_name](num_samples)
    
    random_data_df = pd.DataFrame(random_data, columns=california.feature_names)
    scaler = StandardScaler()
    random_data_scaled = scaler.fit_transform(random_data_df)
    
    generated_targets = model.predict(random_data_scaled)
    
    new_X = pd.DataFrame(random_data, columns=california.feature_names)
    new_y = pd.Series(generated_targets, name='Generated_Price')
    
    new_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    new_model.fit(new_X, new_y)
    
    y_pred_new_model = new_model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))
    
    mse_new_model = mean_squared_error(y_test, y_pred_new_model)
    r2_new_model = r2_score(y_test, y_pred_new_model)
    
    return combination, mse_new_model, r2_new_model, new_X, new_y

# Параллельное выполнение
results = Parallel(n_jobs=-1)(delayed(evaluate_combination)(comb) for comb in all_combinations)

# Поиск лучшего результата
for combination, mse_new_model, r2_new_model, new_X, new_y in results:
    if mse_new_model < best_mse:
        print(f"найдены данные получше... mse: {mse_new_model} r2:{r2_new_model}")
        best_mse = mse_new_model
        best_r2 = r2_new_model
        best_distribution = combination
        best_new_X = new_X
        best_new_y = new_y

print(f"Best distribution combination: {best_distribution}, Best MSE: {best_mse:.2f}, Best R^2: {best_r2:.2f}")

# Обучение и оценка модели с наилучшей комбинацией распределений
best_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
best_model.fit(best_new_X, best_new_y)

y_pred_best_model = best_model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))

plt.scatter(y_test, y_pred_best_model)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices by Best Model')
plt.title('Actual vs Predicted Prices by Best Model')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed


class ModelExtractor:
    model_executor = None

    # Различные виды распределений
    distributions = {
        'uniform': lambda num_samples: np.random.rand(num_samples),
        'normal': lambda num_samples: np.random.randn(num_samples),
        'exponential': lambda num_samples: np.random.exponential(scale=1.0, size=num_samples),
        'gamma': lambda num_samples: np.random.gamma(shape=2.0, scale=1.0, size=num_samples)
    }

    # Все возможные комбинации распределений для каждого столбца
    distribution_names = list(distributions.keys())
    all_combinations = list(itertools.product(distribution_names, repeat=X_train.shape[1]))


    num_samples = 1000
    best_mse = float('inf')
    best_r2 = float('-inf')
    best_distribution = None
    best_new_X = None
    best_new_y = None
    
    def __init__(self,model_type):
        if model_type == "random_forest":
            self.model_executor = RandomForestRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        if model_type == "gradient_boosting":
            self.model_executor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        
        
    # Функция для генерации данных и оценки модели
    def evaluate_combination(self, combination):
        random_data = np.zeros((num_samples, X_train.shape[1]))
        
        for col_idx, dist_name in enumerate(combination):
            random_data[:, col_idx] = distributions[dist_name](num_samples)
        
        random_data_df = pd.DataFrame(random_data, columns=california.feature_names)
        scaler = StandardScaler()
        random_data_scaled = scaler.fit_transform(random_data_df)
        
        generated_targets = model.predict(random_data_scaled)
        
        new_X = pd.DataFrame(random_data, columns=california.feature_names)
        new_y = pd.Series(generated_targets, name='Generated_Price')
        
        new_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        new_model.fit(new_X, new_y)
        
        y_pred_new_model = new_model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))
        
        mse_new_model = mean_squared_error(y_test, y_pred_new_model)
        r2_new_model = r2_score(y_test, y_pred_new_model)
        
        return combination, mse_new_model, r2_new_model, new_X, new_y

    def fit(self, x, y):
        # Параллельное выполнение
        results = Parallel(n_jobs=-1)(delayed(self.evaluate_combination)(comb) for comb in self.all_combinations)
        
        # Поиск лучшего результата
        for combination, mse_new_model, r2_new_model, new_X, new_y in results:
            if mse_new_model < best_mse:
                print(f"найдены данные получше... mse: {mse_new_model} r2:{r2_new_model}")
                self.best_mse = mse_new_model
                self.best_r2 = r2_new_model
                self.best_distribution = combination
                self.best_new_X = new_X
                self.best_new_y = new_y
        
        print(f"Best distribution combination: {self.best_distribution}, Best MSE: {self.best_mse:.2f}, Best R^2: {self.best_r2:.2f}")
        
        # Обучение и оценка модели с наилучшей комбинацией распределений
        self.model_executor.fit(best_new_X, best_new_y)
        
        y_pred_best_model = self.model_executor.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))
        self.model_executor.fit(x, y)


# Загрузка данных
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='Price')

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение первой модели
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train_scaled, y_train)

# Различные виды распределений
distributions = {
    'uniform': lambda num_samples: np.random.rand(num_samples),
    'normal': lambda num_samples: np.random.randn(num_samples),
    'exponential': lambda num_samples: np.random.exponential(scale=1.0, size=num_samples),
    'gamma': lambda num_samples: np.random.gamma(shape=2.0, scale=1.0, size=num_samples)
}

num_samples = 1000
best_mse = float('inf')
best_r2 = float('-inf')
best_distribution = None
best_new_X = None
best_new_y = None

# Все возможные комбинации распределений для каждого столбца
distribution_names = list(distributions.keys())
all_combinations = list(itertools.product(distribution_names, repeat=X_train.shape[1]))

# Функция для генерации данных и оценки модели
def evaluate_combination(combination):
    random_data = np.zeros((num_samples, X_train.shape[1]))
    
    for col_idx, dist_name in enumerate(combination):
        random_data[:, col_idx] = distributions[dist_name](num_samples)
    
    random_data_df = pd.DataFrame(random_data, columns=california.feature_names)
    scaler = StandardScaler()
    random_data_scaled = scaler.fit_transform(random_data_df)
    
    generated_targets = model.predict(random_data_scaled)
    
    new_X = pd.DataFrame(random_data, columns=california.feature_names)
    new_y = pd.Series(generated_targets, name='Generated_Price')
    
    new_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    new_model.fit(new_X, new_y)
    
    y_pred_new_model = new_model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))
    
    mse_new_model = mean_squared_error(y_test, y_pred_new_model)
    r2_new_model = r2_score(y_test, y_pred_new_model)
    
    return combination, mse_new_model, r2_new_model, new_X, new_y

# Параллельное выполнение
results = Parallel(n_jobs=-1)(delayed(evaluate_combination)(comb) for comb in all_combinations)

# Поиск лучшего результата
for combination, mse_new_model, r2_new_model, new_X, new_y in results:
    if mse_new_model < best_mse:
        print(f"найдены данные получше... mse: {mse_new_model} r2:{r2_new_model}")
        best_mse = mse_new_model
        best_r2 = r2_new_model
        best_distribution = combination
        best_new_X = new_X
        best_new_y = new_y

print(f"Best distribution combination: {best_distribution}, Best MSE: {best_mse:.2f}, Best R^2: {best_r2:.2f}")

# Обучение и оценка модели с наилучшей комбинацией распределений
best_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
best_model.fit(best_new_X, best_new_y)

y_pred_best_model = best_model.predict(pd.DataFrame(X_test_scaled, columns=california.feature_names))

plt.scatter(y_test, y_pred_best_model)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices by Best Model')
plt.title('Actual vs Predicted Prices by Best Model')
plt.show()

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

class ModelRecreator:
    def __init__(self, model, num_samples=1000):
        self.model = model
        self.num_samples = num_samples
        self.distributions = {
            'uniform': lambda num_samples: np.random.rand(num_samples),
            'normal': lambda num_samples: np.random.randn(num_samples),
            'exponential': lambda num_samples: np.random.exponential(scale=1.0, size=num_samples),
            # 'gamma': lambda num_samples: np.random.gamma(shape=2.0, scale=1.0, size=num_samples),
            # 'beta': lambda num_samples: np.random.beta(a=2.0, b=5.0, size=num_samples)
        }
        self.best_mse = float('inf')
        self.best_r2 = float('-inf')
        self.best_distribution = None
        self.best_new_X = None
        self.best_new_y = None

    def generate_correlated_data(self, base_data):
        """Создание коррелированных данных на основе базовых данных."""
        correlation_matrix = np.random.rand(base_data.shape[1], base_data.shape[1])
        correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2
        np.fill_diagonal(correlation_matrix, 1)  # Диагональные элементы - 1

        # Генерация случайных данных и коррекция с использованием матрицы корреляции
        random_data = np.random.multivariate_normal(mean=np.zeros(base_data.shape[1]), cov=correlation_matrix, size=self.num_samples)
        return pd.DataFrame(random_data, columns=base_data.columns)

    def evaluate_combination(self, combination, X_test_scaled, y_test):
        """Оценка модели на основе генерации данных."""
        random_data = np.zeros((self.num_samples, X_test_scaled.shape[1]))

        for col_idx, dist_name in enumerate(combination):
            random_data[:, col_idx] = self.distributions[dist_name](self.num_samples)

        # Создание коррелированных данных
        random_data_df = pd.DataFrame(random_data, columns=california.feature_names)
        correlated_data_df = self.generate_correlated_data(random_data_df)

        scaler = StandardScaler()
        correlated_data_scaled = scaler.fit_transform(correlated_data_df)

        generated_targets = self.model.predict(correlated_data_scaled)

        new_X = correlated_data_df
        new_y = pd.Series(generated_targets, name='Generated_Price')

        new_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        new_model.fit(new_X, new_y)

        y_pred_new_model = new_model.predict(X_test_scaled)

        mse_new_model = mean_squared_error(y_test, y_pred_new_model)
        r2_new_model = r2_score(y_test, y_pred_new_model)

        return combination, mse_new_model, r2_new_model, new_X, new_y

    def recreate_model(self, X_train, X_test_scaled, y_test):
        """Восстановление модели на основе сгенерированных данных."""
        distribution_names = list(self.distributions.keys())
        all_combinations = list(itertools.product(distribution_names, repeat=X_train.shape[1]))

        results = Parallel(n_jobs=-1)(delayed(self.evaluate_combination)(comb, X_test_scaled, y_test) for comb in all_combinations)

        for combination, mse_new_model, r2_new_model, new_X, new_y in results:
            if mse_new_model < self.best_mse:
                print(f"найдены данные получше... mse: {mse_new_model} r2: {r2_new_model}")
                self.best_mse = mse_new_model
                self.best_r2 = r2_new_model
                self.best_distribution = combination
                self.best_new_X = new_X
                self.best_new_y = new_y

        print(f"Best distribution combination: {self.best_distribution}, Best MSE: {self.best_mse:.2f}, Best R^2: {self.best_r2:.2f}")

        best_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        best_model.fit(self.best_new_X, self.best_new_y)

        y_pred_best_model = best_model.predict(X_test_scaled)

        plt.scatter(y_test, y_pred_best_model)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', lw=2)
        plt.xlabel('Actual Prices')
        plt.ylabel('Predicted Prices by Best Model')
        plt.title('Actual vs Predicted Prices by Best Model')
        plt.show()

# Пример использования
california = fetch_california_housing()
X = california.data
y = california.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_e = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_e.fit(X_train_scaled, y_train)

recreator = ModelRecreator(model_e)
recreator.recreate_model(X_train_scaled, X_test_scaled, y_test)


/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0qyns78b3rgq_k73qmd8h0000gn/T/ipykernel_65905/2749222552.py:40: RuntimeWarning: covariance is not positive-semidefinite.
/var/folders/dz/t2v0

KeyboardInterrupt: 